In [1]:
import os
from datetime import datetime
#from datetime import date
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv, find_dotenv
import pandas as pd
from numpy import int16
from pathlib import Path


In [2]:
load_dotenv(find_dotenv())

USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")
query = '''
SELECT 
    d.case_id,
    p.patient_code as dreams_code,
    d.organisation,
    d.case_id AS id,
    d.e__telefn AS phone,
    IF(dm.case_id IS NOT NULL, 1, 0) AS not_selectable,
    d.a_non_patisipan_an AS first_name,
    d.b_siyati AS last_name,
    d.nan_ki_dat_ou_fet AS dob,
    d.a1_dat_entvyou_a_ft_jjmmaa_egz_010817 AS interview_date,
    d.d_adrs AS adress,
    d.f_komin AS commune,
    d.g_seksyon_kominal AS commune_section,
    d.b1_non_moun_mennen_entvyou_a AS interviewer_firstname,
    d.c1_siyati_moun_ki_f_entvyou_a AS interviewer_lastname,
    d.paran_ou_vivan AS is_your_parent_alive,
    d.i_non_manman AS mothers_name,
    d.j_non_papa AS fathers_name,
    d.total,
    d.k_reskonsab_devan_lalwa AS who_is_your_law_parent,
    dg.name AS group_name,
    dg.id AS id_group,
    dh.name as hub_name,
    dh.id as id_hub,
    
    IF(COALESCE(d.a_ske_ou_lekl, d.tranche_age_6a_15_19) = 0,
        'oui',
        'no') AS eske_ou_lekol_deja,
    CASE
        WHEN d.c61_depi_ki_l_ou_pa_al_lekl_ank = 1 THEN 'Ane sa a'
        WHEN d.c61_depi_ki_l_ou_pa_al_lekl_ank = 2 THEN 'Ane pase'
        WHEN d.c61_depi_ki_l_ou_pa_al_lekl_ank = 3 THEN 'Sa gen plis pase dezan'
        WHEN d.c61_depi_ki_l_ou_pa_al_lekl_ank = 4 THEN 'Pa t janm al lekòl'
    END AS depi_kile_ou_pa_al_lekol_with_label,
    d.c61_depi_ki_l_ou_pa_al_lekl_ank,
    d.c6_ske_ou_te_oblije_double_deja,
    case
    WHEN d.c6b_kils_ki_peye_lekl_ou_10_14='000' then 'oganizasyon'
    WHEN d.c6b_kils_ki_peye_lekl_ou_10_14='00' then 'Manman/papa'
    WHEN d.c6b_kils_ki_peye_lekl_ou_10_14='0' then 'Lòt fanmi'
    WHEN d.c6b_kils_ki_peye_lekl_ou_10_14='5' then 'Li menm'
    WHEN d.c6b_kils_ki_peye_lekl_ou_10_14='05' then 'Zanmi'
    WHEN d.c6b_kils_ki_peye_lekl_ou_10_14='001' then 'Pa konnen'
    WHEN d.c6b_kils_ki_peye_lekl_ou_10_14='8' then 'Patnè seksyèl'
    WHEN d.c6b_kils_ki_peye_lekl_ou_10_14='lt_presize' then d.lot_ki_peye
    WHEN d.c6b_Kiles_ki_peye_lekol_ou_Tranche_15_19='000' then 'oganizasyon'
    WHEN d.c6b_Kiles_ki_peye_lekol_ou_Tranche_15_19='0' then 'Manman/papa'
    WHEN d.c6b_Kiles_ki_peye_lekol_ou_Tranche_15_19='00' then 'Lot Fanmi'
    WHEN d.c6b_Kiles_ki_peye_lekol_ou_Tranche_15_19='3' then 'Li menm'
    WHEN d.c6b_Kiles_ki_peye_lekol_ou_Tranche_15_19='03' then 'Zanmi'
    WHEN d.c6b_Kiles_ki_peye_lekol_ou_Tranche_15_19='01' then 'Pa konnen'
    WHEN d.c6b_Kiles_ki_peye_lekol_ou_Tranche_15_19='6' then 'Patnè seksyèl'
    WHEN d.c6b_Kiles_ki_peye_lekol_ou_Tranche_15_19='pran_not_ou_bay_la' then d.lot_presize_15_19
    
    end as kiyes_ki_peye_lekol_pou_ou,
    TIMESTAMPDIFF(YEAR,
        d.nan_ki_dat_ou_fet,
        d.a1_dat_entvyou_a_ft_jjmmaa_egz_010817) AS age,
		if(
        TIMESTAMPDIFF(YEAR,
        d.nan_ki_dat_ou_fet,
        d.a1_dat_entvyou_a_ft_jjmmaa_egz_010817) between 10 and 18,1,-50 ) + (
        CASE d.a7_ak_kils_w_ap_viv_10_14 
        WHEN '0' THEN 2
        WHEN '1' THEN 4
        WHEN '001' THEN 4
        WHEN '03' THEN 4
        WHEN '8' THEN -50
        WHEN '6' THEN -50
        WHEN 'lt_presize' THEN 2
        ELSE 0
        END )+(
        CASE d.a7_Ak_kiles_w_ap_viv_15_19
			WHEN '0' THEN 2
			WHEN '1' THEN 4
			WHEN '01' THEN 4
			WHEN '02' THEN 4
			WHEN '5' THEN -50
			WHEN '3' THEN -50
			WHEN 'lt_presize'  THEN 2
            ELSE 0
        END
        ) +(
        CASE d.a7_Ak_kiles_w_ap_viv_17_18
			WHEN '0' THEN 2
			WHEN '1' THEN 4
			WHEN '01' THEN 4
			WHEN '2' THEN 4
			WHEN '3' THEN -50
			WHEN '03' THEN -50
			WHEN 'lt_presize'  THEN 2
            ELSE 0
        END
        )+(
        CASE d.a8_eske_gen_yon_moun_ou_santi_ou_ka_al_kote_
			WHEN '0' THEN 0
			WHEN '2' THEN 4
            ELSE 0
        END
        )+(
        CASE d.a9_ou_gen_lot_kote_ou_pa_santi_ou_ansekirite
			WHEN '1' THEN 2
			WHEN '0' THEN 0
            ELSE 0
        END
        )+(
        CASE d.b9_list_kote_ou_pa_an_sekirite
			WHEN '4' THEN 8
			WHEN '1' THEN 4
            WHEN '3' THEN 4
            WHEN '2' THEN 4
            WHEN '04' THEN 4
            WHEN '0001' THEN 2
            ELSE 0
        END
        )+(
        CASE d.a17
			WHEN '0' THEN 0
			WHEN '4' THEN 8
            WHEN '04' THEN 8
            WHEN '004' THEN 8
            ELSE 0
        END
        )+(
        CASE d.eske_ou_konn_bwe_alkol
			WHEN 'oui' THEN 3
            ELSE 0
        END
        )+(
        CASE d.a21_ske_ou_bw_alkl_osnon_itilize_lt_dwg_10_14
			WHEN '3' THEN 6
            WHEN '5' THEN 8
            ELSE 0
        END
        )+(
        CASE d.a21_nan_ki_frenkans_ou_itilize_alkol
			WHEN '2' THEN 6
            WHEN '5' THEN 8
            ELSE 0
        END
        ) +(
        CASE d.eske_ou_konn_itilize_dwog
			WHEN 'oui' THEN 3
            ELSE 0
        END
        )+(
        CASE d.ske_ou_konn_itilize_dwg_10_14
			WHEN '3' THEN 6
            WHEN '5' THEN 8
            ELSE 0
        END
        )+(
        CASE d.ske_ou_konn_itilize_dwg_15_19
			WHEN '2' THEN 6
            WHEN '5' THEN 8
            ELSE 0
        END
        )  as kpi_score
FROM
    caris_db.dreams_surveys_data d
        LEFT JOIN
    caris_db.dream_member dm ON dm.case_id = d.case_id
    left join patient p on p.id=dm.id_patient
        LEFT JOIN
    caris_db.dream_group dg ON dg.id = dm.id_group
    LEFT JOIN caris_db.dream_hub dh on dh.id=dg.id_dream_hub
WHERE
    d.total >= 14 group by d.case_id
'''

eligibility = pd.read_sql_query(query,engine,parse_dates=True)
# get the test excel file from Query
engine.dispose()

In [3]:
eligibility.to_excel('./eligibility.xlsx',index=False,na_rep="NULL")